In [27]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [28]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
lending_data_df = pd.read_csv(Path("Resources/lending_data.csv"))

# Review the DataFrame
display(lending_data_df.head())
display(lending_data_df.tail())

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700,7.672,52800,0.431818,5,1,22800,0
1,8400,6.692,43600,0.311927,3,0,13600,0
2,9000,6.963,46100,0.349241,3,0,16100,0
3,10700,7.664,52700,0.430740,5,1,22700,0
4,10800,7.698,53000,0.433962,5,1,23000,0


,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
77531,19100,11.261,86600,0.653580,12,2,56600,1
77532,17700,10.662,80900,0.629172,11,2,50900,1
77533,17600,10.595,80300,0.626401,11,2,50300,1
77534,16300,10.068,75300,0.601594,10,2,45300,1
77535,15600,9.742,72300,0.585062,9,2,42300,1


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [29]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = lending_data_df['loan_status']

# Separate the X variable, the features
x = lending_data_df.drop(columns=['loan_status'])

In [30]:
# Review the y variable Series
y.head()

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [31]:
# Review the X variable DataFrame
x.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700,7.672,52800,0.431818,5,1,22800
1,8400,6.692,43600,0.311927,3,0,13600
2,9000,6.963,46100,0.349241,3,0,16100
3,10700,7.664,52700,0.430740,5,1,22700
4,10800,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [32]:
# Check the balance of our target values
y.value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [33]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=1, stratify=y)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [34]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
logistic_regression_model_default_risk = LogisticRegression(random_state=1)

# Fit the model using training data
credit_risk_model = logistic_regression_model_default_risk.fit(X_train, y_train)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [35]:
# Make a prediction using the testing data
testing_credit_risk_predictions = credit_risk_model.predict(X_test)
print(len(testing_credit_risk_predictions))

19384


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [36]:
# Print the balanced_accuracy score of the model
logistic_accuracy_score = balanced_accuracy_score(y_test, testing_credit_risk_predictions)
print(logistic_accuracy_score)

0.9442676901753825


In [37]:
# Generate a confusion matrix for the model
logistic_confusion_matrix = confusion_matrix(y_test, testing_credit_risk_predictions)
print(logistic_confusion_matrix)

[[18679    80]
 [   67   558]]


In [38]:
# Print the classification report for the model
logistic_classification_report = classification_report(y_test, testing_credit_risk_predictions)
print(logistic_classification_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18759
           1       0.87      0.89      0.88       625

    accuracy                           0.99     19384
   macro avg       0.94      0.94      0.94     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The logistic regression model predicts a healthy (low-risk) loan with 100% precision, while it predicts a high-risk loan with a lower precision at 87%. In general, the logistic regression model is good at predicting whether a loan may default (not a healthy loan, or is a high risk loan) because of its high balanced accuracy at 94.4% and somewhat high F-1 and recall scores. If the bank is still getting a high precision and recall on the test dataset (even if they are lower scores than for the train dataset), it is a good indication about how well the model is likely to perform in real life.

In summary, the accuracy of the logistic regression model seems to be good enough to start exploring this algorithm in a bank setting for assessing the creditworthiness of borrowers; however, it may be prudent for the bank to start running a pilot with new data to assess the model's reliability on data the model has not "seen" yet.

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [39]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# Assign a random_state parameter of 1 to the model
random_over_sample = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
x_resample, y_resample = random_over_sample.fit_resample(X_train, y_train)

In [40]:
# Count the distinct values of the resampled labels data - loan StatusView
y_resample.value_counts()

0    56277
1    56277
Name: loan_status, dtype: int64

In [41]:
# View the distinct values of the resampled X labels data
x_resample.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
count,112554.000000,112554.000000,112554.000000,112554.000000,112554.000000,112554.000000,112554.000000
mean,14021.289337,9.083750,66081.736766,0.504699,7.613323,1.245891,36081.736766
std,4750.474442,2.018673,18998.687571,0.145852,4.280098,1.020908,18998.687571
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000
25%,9500.000000,7.147000,47900.000000,0.373695,4.000000,0.000000,17900.000000
50%,13900.000000,9.041000,65700.000000,0.543379,8.000000,1.000000,35700.000000
75%,18600.000000,11.042000,84500.000000,0.644970,12.000000,2.000000,54500.000000
max,23500.000000,13.126000,104100.000000,0.711816,16.000000,3.000000,74100.000000


### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [42]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
logistic_classifier = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using the resampled training data
logistic_classifier.fit(x_resample, y_resample)

# Make a prediction using the testing data
test_predictions = logistic_classifier.predict(X_test)
pd.DataFrame({"Prediction": test_predictions, "Actual": y_test})

,Prediction,Actual
36831,0,0
75818,1,1
36563,0,0
13237,0,0
43292,0,0
...,...,...
38069,0,0
36892,0,0
5035,0,0
40821,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [43]:
# Print the balanced_accuracy score of the model 
predicted_logistic_accuracy_score2 = balanced_accuracy_score(y_test, test_predictions)
print(predicted_logistic_accuracy_score2)

0.9959744975744975


In [44]:
# Generate a confusion matrix for the model
predicted_logistic_confusion_matrix2 = confusion_matrix(y_test, test_predictions)
print(predicted_logistic_confusion_matrix2)

[[18668    91]
 [    2   623]]


In [45]:
# Print the classification report for the model
predicted_logistic_classification_report2 = classification_report(y_test, test_predictions)
print(predicted_logistic_classification_report2)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18759
           1       0.87      1.00      0.93       625

    accuracy                           1.00     19384
   macro avg       0.94      1.00      0.96     19384
weighted avg       1.00      1.00      1.00     19384



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The resampled logistic regression model (using the RandomOverSampler module) predicts a healthy/low-risk loan with 100% precision, while it predicts a high-risk loan with a lower precision at 87%; both precision percentages were the same as in the original logistic regression model. However, the balanced accuracy for the resampled logistic regression model is 99.6%, in comparison to 94.4% for the original logistic regression model.  Similarly, the F-1 and recall scores were higher in the resampled logistic regression model.